In [1]:
from ipsl_dataset import IPSL_DCPP
import torch
train = IPSL_DCPP('train',generate_statistics=False,lead_time_months=5*12)
train_dataloader = torch.utils.data.DataLoader(train,batch_size=1,shuffle=True,num_workers=1,)


In [2]:
from pangu import PanguWeather
model = PanguWeather(soil=True)

/gpfsdswork/projects/rech/mlr/udy16au/miniconda3/envs/env_gpu5/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/gpfsdswork/projects/rech/mlr/udy16au/miniconda3/envs/env_gpu5/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [3]:
from hydra import compose, initialize
from omegaconf import OmegaConf


with initialize(version_base=None, config_path="conf"):
    cfg = compose(config_name="config")


In [4]:
train_dataloader = torch.utils.data.DataLoader(IPSL_DCPP('train',cfg.experiment.lead_time_months),batch_size=1,shuffle=False,num_workers=1)
batch = next(iter(train_dataloader))

In [ ]:
out = model.forward(batch)